# Individual solution validation via the REST-API of the Solution Validator Service

_Note: For full compatibility, use Python >= 3.6 to run this notebook._

The REST-API of the Solution Validator can be used to check
* whether a solution respects all required planning rules (and if not, where exactly the violoations are)
* the objectiveValue of a solution

It thus provides a shortcut to the complete submission-procedure (where you are required to submit a full set of solutions for all problem instances) to validate individual solutions.

This makes frequent testing and tuning of your algorithm much easiere.

_However: Please observe a limit of at most one (1) validation per minute in order not to overwhelm the service_

## Examples

_Note: The following code is also collected in_ [this](validate_solution.py) _script_

Setup config:

In [1]:
import requests

AUTH = ('ebrown', 'plutonium94')
SCENARIO_UPLOAD_ENDPOINT = "https://fluxer.app.sbb.ch/backend/verkehrsplan/uploadVerkehrsplan"
SOLUTION_VALIDATION_ENDPOINT = "https://fluxer.app.sbb.ch/backend/loesung-validator/validateFile"

Specify the files to use. Validating a solution only makes sense when also specifying which problem instance the solution is intended for. So we specify both the problem instance and the solution. Both must be available as a JSON file.

In [2]:
scenario = "samples/sample_scenario_simple.json"
solution = "samples/sample_scenario_simple_solution.json"

Upload the scenario file first.

### ToDo: We must make sure that the has in the scenario file is correct. Put in an assertion

In [4]:
scenario_file = {"verkehrsplan": open(scenario, 'rb')}
upload_response = requests.post(SCENARIO_UPLOAD_ENDPOINT, files=scenario_file, auth=AUTH)
upload_response

<Response [200]>

Now we can validate

In [5]:
solution_file = {"loesung": open(solution, 'rb')}
validation_response = requests.post(SOLUTION_VALIDATION_ENDPOINT, files=solution_file, auth=AUTH)
validation_result = validation_response.json()

Inspect the response. The rule violations are collected in the attribute `regelVerletzungen`

In [6]:
warnings = [x for x in validation_result["regelVerletzungen"] if x["severity"] == "warning"]
errors = [x for x in validation_result["regelVerletzungen"] if x["severity"] == "error"]
print(f"solution has {len(warnings)} warnings")
print(f"solution has {len(errors)} errors")

solution has 0 warnings
solution has 0 errors


### Example: Warning "wrong Hash in solution"

The following solution has a wrong solution hash. This causes a warning, but it is irrelevant. Also, the solution is not penalized because of this. objValue is zero.

__you may safely ignore all solution-hash warnings in your solutions__

In [7]:
solution = "samples/sample_scenario_simple_solution_warningHash.json"
solution_file = {"loesung": open(solution, 'rb')}
validation_response = requests.post(SOLUTION_VALIDATION_ENDPOINT, files=solution_file, auth=AUTH)
validation_result = validation_response.json()

assert len(validation_result["regelVerletzungen"]) == 1


### Example: Warning "delayed arrival"

In [9]:
solution = "samples/sample_scenario_simple_solution_delayed_arrival.json"
solution_file = {"loesung": open(solution, 'rb')}
validation_response = requests.post(SOLUTION_VALIDATION_ENDPOINT, files=solution_file, auth=AUTH)
validation_result = validation_response.json()

from pprint import pprint
pprint(validation_result)

{'details': {'Fahrwegpenalty': '0.0', 'Verspaetungsminuten': '8.65'},
 'loesungHash': -250799109,
 'objectiveValue': 8.65,
 'regelVerletzungen': [{'message': 'Austrittszeit 08:50:39 nach ausMax 08:42 '
                                   'für Zugfahrtabschnitt mit FAB-Id '
                                   '"PF_SA/IC#17" und Abschnittskennzeichen '
                                   '"MELS" in fA "IC/559-001/PF-SA"',
                        'severity': 'warning'}],
 'requestUuid': None}


### Example: Errors "early departure" and "resource occupation conflict"

This solution has actual _errors_. It will _not_ be accepted as a feasible solution by the grader. There are two errors:
* The service intentions "EC/163-001/PF-SA" and "IC/913-001/PF-SA" overtake each other on the route section PF_SA/IC#17. This (obviously) violates the separation constraints for the associated resource "MELS_SA"
* Service Intention EC/163-001/PF-SA enters into route section PF_SA/IC#1 at 09:00:30. This is _before_ the earliest allowed entry time of 09:02:30.

Note: In addition, there is also a warning because train IC/913-001/PF-SA arrives too late.

In [8]:
solution = "samples/sample_scenario_simple_solution_errors.json"
solution_file = {"loesung": open(solution, 'rb')}
validation_response = requests.post(SOLUTION_VALIDATION_ENDPOINT, files=solution_file, auth=AUTH)
validation_result = validation_response.json()
errors = [x for x in validation_result['regelVerletzungen'] if x['severity'] == 'error']

print(f"There are {len(errors)} errors" + "\n")

for x in validation_result["regelVerletzungen"]:
    if x["severity"] == 'error':
        print(x["message"] + '\n')

There are 2 errors

Belegungskonflikt (Zugfolgezeit gleiche Richtung: MELS_SA), Zugfolgezeit[s]: 180, Ressource: "MELS_SA", fAs: "EC/163-001/PF-SA" / "IC/913-001/PF-SA", FABs: "PF_SA/IC#17" / "PF_SA/IC#17", Zeiten ein-aus: 09:33:36.600-09:35:39 / 09:00:36.600-09:40:39

Eintrittszeit 09:00:30 vor einMin 09:02:30 für Zugfahrtabschnitt mit FAB-Id "PF_SA/IC#1" und Abschnittskennzeichen "PF" in fA "EC/163-001/PF-SA"

